In [ ]:
!pip list

In [2]:
!pip install pytorch-lightning 

   ---------------------------------------- 0.0/776.9 kB ? eta -:--:--
   --- ------------------------------------ 61.4/776.9 kB 3.4 MB/s eta 0:00:01
   ----------------- ---------------------- 337.9/776.9 kB 4.2 MB/s eta 0:00:01
   ------------------------------ --------- 583.7/776.9 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------  768.0/776.9 kB 4.9 MB/s eta 0:00:01
   ---------------------------------------- 776.9/776.9 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/806.1 kB ? eta -:--:--
   --------- ----------------------------- 204.8/806.1 kB 13.0 MB/s eta 0:00:01
   ------------------ --------------------- 368.6/806.1 kB 5.8 MB/s eta 0:00:01
   ------------------ --------------------- 368.6/806.1 kB 5.8 MB/s eta 0:00:01
   ------------------ --------------------- 368.6/806.1 kB 5.8 MB/s eta 0:00:01
   --------------------------------- ------ 675.8/806.1 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------  788.5/806.1 kB 

In [3]:
!pip  install kss

     ---------------------------------------- 0.0/79.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/79.1 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/79.1 kB ? eta -:--:--
     ------------------- ------------------ 41.0/79.1 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 79.1/79.1 kB 550.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/131.3 kB ? eta -:--:--
     ----------------------------------- -- 122.9/131.3 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 131.3/131.3 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/26.4 MB ? eta -:--:--
     ---------------------------------------- 0.3/26.4 MB 6.3 MB/s eta 0:00:05
      --------------------------------------- 0.6/26.4 MB 7.4 MB/s eta 0:00:04
     - -------------------------------------- 0.9/26.4 MB 

In [5]:
!pip install seaborn

   ---------------------------------------- 0.0/294.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.6 kB ? eta -:--:--
   --------- ----------------------------- 71.7/294.6 kB 975.2 kB/s eta 0:00:01
   ---------------------------------------- 294.6/294.6 kB 2.6 MB/s eta 0:00:00


In [2]:
import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl
import torchmetrics 
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import kss

c:\Users\echoi\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
!pip list

Package                       Version
----------------------------- ------------
absl-py                       2.0.0
aiohttp                       3.9.0
aiosignal                     1.3.1
altair                        4.1.0
annotated-types               0.6.0
anyio                         3.7.1
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
astor                         0.8.1
asttokens                     2.4.0
astunparse                    1.6.3
async-timeout                 4.0.3
attrs                         23.1.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
base58                        2.1.1
beautifulsoup4                4.12.2
bert-extractive-summarizer    0.10.1
bleach                        4.1.0
blinker                       1.7.0
blis                          0.7.11
boto3                         1.33.6
botocore                      1.33.6
breadability                  0.1.20
brotlipy 

In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Seed set to 42


42

In [4]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 10
BATCH_SIZE = 4

In [5]:
pwd

'c:\\Users\\echoi\\project\\kpfbertsum'

In [11]:
DATA_TRAIN_PATH = 'data/train_original.json'
df = pd.read_json(DATA_TRAIN_PATH)
df = df.dropna()
len(df)#, len(val_df)

243983

In [12]:
DATA_TEST_PATH = 'data/valid_original.json'
test_df = pd.read_json(DATA_TEST_PATH)
test_df = test_df.dropna()
len(test_df)

30122

In [8]:
train_df, val_df = train_test_split(df, test_size=0.05)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
train_df.shape, val_df.shape, test_df.shape

((231783, 3), (12200, 3), (30122, 3))

In [10]:
# # test setting all data downsize
# downsize = 2000
# train_df = train_df[:downsize]
# test_df = test_df[:downsize//10]
# val_df = val_df[:downsize//10]

In [92]:
train_df.shape, test_df.shape, val_df.shape

((2000, 3), (200, 3), (200, 3))

In [93]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [94]:
train_df = preprocess_data(train_df)
train_df.head(1)

,media,id,article_original,abstractive,extractive
0,중도일보,353559055,"[금 2, 은 3, 동 2 획득, 올해 최고 성적인 종합 2위 달성, 제14회 대통...","서산시청 사격팀은 '제14회 대통령경호처장기 전국사격대회에서 금메달 2개, 은메달 ...","[2, 3, 4]"


In [95]:
i = 8
print('===== 본    문 =====')
for idx, str in enumerate(train_df['article_original'][i]):
    print(idx,':',str)
print('===== 요약정답 =====')
print(train_df['extractive'][i])
print('===== 추출본문 =====')
print('1 :', train_df['article_original'][i][train_df['extractive'][i][0]])
print('2 :', train_df['article_original'][i][train_df['extractive'][i][1]])
print('3 :', train_df['article_original'][i][train_df['extractive'][i][2]])
print('===== 생성본문 =====')
print(train_df['abstractive'][i])

===== 본    문 =====
0 : 특성화고 80명, 보훈대상자 20명
1 : 올 한해 총 750명 채용 계획
2 : 우리은행(은행장 손태승)은 일자리 창출과 우수 인재 확보를 위해 100여 명의 특별채용을 진행하고 있다고 23일 밝혔다.
3 : 우리은행은 이번 특별채용을 통해 특성화고 출신 80명, 국가보훈대상자 20명을 채용한다.
4 : 올해 특성화고 출신 채용 인원은 80명으로 이는 전년의 60명 대비 33% 증가한 규모다.
5 : 우리은행은 지난 8년간 금융권 최대 규모인 778명의 특성화고 출신 행원을 채용했다.
6 : 특성화고 출신 채용은 교육부와 협업을 통해 학교장 추천을 받은 고3 학생을 대상으로 진행되며, 현재 서류전형 합격자를 대상으로 1차 면접이 진행되고 있다.
7 : 올해 보훈 특별채용 인원은 총 40명으로 이는 전년 대비 100% 증가한 규모다.
8 : 우리은행은 지난 상반기에 국가보훈대상자 20명을 채용했으며, 하반기 역시 20명을 채용한다.
9 : 서류전형 합격자를 대상으로 23일 1차 면접을 진행 중이다.
10 : 보훈 특별채용과 관련 손태승 우리금융그룹회장 겸 우리은행장은 지난 7월 서울지방보훈청 주최로 진행된 '2019년 호국보훈의 달 대외 유공인사 국가보훈처장 감사패 전수식'에서 국가보훈처장 감사패를 받았다.
11 : 국가보훈처는 매년 6월 호국보훈의 달을 맞아 보훈대상자 취업에 모범이 되는 기관의 장에게 국가보훈처장감사패를 수여한다.
12 : 한편, 우리은행은 금융권 고용창출을 위해 올해 750명을 채용할 계획이다.
13 : 현재 상반기 300명 규모의 채용을 완료했으며, 하반기에는 본 특별채용을 포함하여 450명을 채용할 계획이다.
14 : 우리은행 채용 담당자는 "직무능력과 인성을 겸비한 신입행원을 선발하기 위하여 블라인드 면접을 진행하고 있다"며, "우리은행 취업을 위해 부단히 준비해 온 지원자에게 좋은 결과가 있기를 바란다"고 말했다.
===== 요약정답 =====
[2, 12, 14]
===== 추출본문 ===

In [96]:
test_df = preprocess_data(test_df)
test_df.head(1)

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"


In [97]:
val_df = preprocess_data(val_df)
val_df.head(1)

,media,id,article_original,abstractive,extractive
0,전북일보,351014195,"[최명국, 송하진 지사, 방중 주요 성과로 '군산~연운항' 항로 개설 협의 꼽아, ...",송하진 전북도지사가 중국 장쑤성을 방문해 러우 친지앤 당서기와 새만금 산단 5공구 ...,"[3, 7, 8]"


In [98]:
BERT_MODEL_NAME = 'kpfbert-base'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [99]:
class SummDataset(Dataset):

    def __init__(
        self, 
        data: pd.DataFrame, 
        tokenizer: BertTokenizer, 
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'
    
        src = [] # 토크나이징 된 전체 문단
        labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1
        
            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break
    
        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))
            
        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )

In [100]:
class SummDataModule(pl.LightningDataModule):

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = SummDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )
    
        self.val_dataset = SummDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

In [101]:
data_module = SummDataModule(
  train_df,
  test_df,  
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

In [102]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]

In [103]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)
        

In [104]:
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.2, num_inter_layers=2):
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask) 

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores

In [105]:
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x

In [106]:
class MultiHeadedAttention(nn.Module):
    """
    Multi-Head Attention module from
    "Attention is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`.

    Similar to standard `dot` attention but uses
    multiple attention distributions simulataneously
    to select relevant items.

    .. mermaid::

       graph BT
          A[key]
          B[value]
          C[query]
          O[output]
          subgraph Attn
            D[Attn 1]
            E[Attn 2]
            F[Attn N]
          end
          A --> D
          C --> D
          A --> E
          C --> E
          A --> F
          C --> F
          D --> O
          E --> O
          F --> O
          B --> O

    Also includes several additional tricks.

    Args:
       head_count (int): number of parallel heads
       model_dim (int): the dimension of keys/values/queries,
           must be divisible by head_count
       dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)

    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):
        """
        Compute the context vector and the attention vectors.

        Args:
           key (`FloatTensor`): set of `key_len`
                key vectors `[batch, key_len, dim]`
           value (`FloatTensor`): set of `key_len`
                value vectors `[batch, key_len, dim]`
           query (`FloatTensor`): set of `query_len`
                 query vectors  `[batch, query_len, dim]`
           mask: binary mask indicating which keys have
                 non-zero attention `[batch, query_len, key_len]`
        Returns:
           (`FloatTensor`, `FloatTensor`) :

           * output context vectors `[batch, query_len, dim]`
           * one of the attention vectors `[batch, query_len, key_len]`
        """

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context

In [1]:
class Summarizer(pl.LightningModule):

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME) #, return_dict=True)
        
         # BERT 레이어의 파라미터 동결하기
        for param in self.bert.parameters():
            param.requires_grad = False  # BERT 파라미터를 동결함

        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')

        

        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):
        

        mask_src = ~(src == 0) #1 - (src == 0)
        mask_cls = ~(clss == -1) #1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state
        
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        
        

        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            
            loss = (loss * mask_cls.float()).sum() / len(labels)
            
            # print("sent_scores shape:", sent_scores.shape)
            # print("sent_scores dtype:", sent_scores.dtype)
            # print("labels shape:", labels.shape)
            # print("labels dtype:", labels.dtype)
            print("loss:", loss)
            # print("loss shape:", loss.shape)
            # print("loss dtype:", loss.dtype)
        return loss, sent_scores
    
    def step(self, batch):

        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']
        
        loss, sent_scores = self(src, segs, clss, labels)    
        
        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def acc_loss(self, outputs):
        total_loss = 0
        hit_cnt = 0
        for outp in outputs:
            labels = outp['labels'].cpu()
            predictions, idxs = outp['predictions'].cpu().sort()
            loss = outp['loss'].cpu()
            for label, idx in zip(labels, idxs):
                for i in range(1,3):
                    if label[idx[-i-1]] == 1 : 
                        hit_cnt += 1

            total_loss += loss
            
        avg_loss = total_loss / len(outputs)
        acc = hit_cnt / (3*len(outputs)*len(labels))
        
        return acc, avg_loss
        
    def training_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('acc:', acc, 'avg_loss:', avg_loss)
        
        self.log('avg_train_loss', avg_loss, prog_bar=True, logger=True)

    def validation_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('val_acc:', acc, 'avg_val_loss:', avg_loss)
        
        self.log('avg_val_loss', avg_loss, prog_bar=True, logger=True)

    def test_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('test_acc:', acc, 'avg_test_loss:', avg_loss)
        
        self.log('avg_test_loss', avg_loss, prog_bar=True, logger=True)

        return
        
    def configure_optimizers(self):
        
        optimizer = AdamW(self.parameters(), lr=2e-5)

        steps_per_epoch=len(train_df) // BATCH_SIZE
        total_training_steps = steps_per_epoch * N_EPOCHS
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

NameError: name 'pl' is not defined

In [161]:
from transformers import BertModel, BertTokenizer

model_name_or_path = "kpfbert-base"  # Bert 바이너리가 포함된 디렉토리

model = BertModel.from_pretrained(model_name_or_path, add_pooling_layer=False)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)

In [ ]:
# 기존 모델 파라미터 초기화
model.init_weights()

In [162]:
model = Summarizer()

Some weights of BertModel were not initialized from the model checkpoint at kpfbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [163]:
!rm -rf lightning_logs/
!rm -rf checkpoints/

In [164]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 58573), started 1:29:26 ago. (Use '!kill 58573' to kill it.)

In [165]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="avg_val_loss",
    mode="min"
)

In [166]:
logger = TensorBoardLogger("lightning_logs", name="kpfBERT_Summary")


In [167]:
early_stopping_callback = EarlyStopping(monitor='avg_val_loss', patience=3)

In [168]:
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=0,
#     precision=16, #소스 수정 또는 패키지 재설치 필요... 런타임 에러.
    progress_bar_refresh_rate=30
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [169]:
trainer.fit(model, data_module)


  | Name      | Type                  | Params
----------------------------------------------------
0 | bert      | BertModel             | 114 M 
1 | ext_layer | ExtTransformerEncoder | 11.0 M
2 | loss      | BCELoss               | 0     
----------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.230   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_scores dtype: torch.float32
labels shape: torch.Size([4, 512])
labels dtype: torch.float32
sent_scores shape: torch.Size([4, 512])
sent_sco

KeyboardInterrupt: 

In [118]:
for name, param in model.named_parameters():
    if param.requires_grad == False:
        print(f"{name}: requires_grad={param.requires_grad}")

NameError: name 'sent_scores' is not defined